# Расчет данных по группам проектов

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()

# Общие параметры для заданий

Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять только в одном месте.

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по географии, в нашем случае он не требуется 
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Создаем словарь для нужных нам типов пользования интернетом
ut = {'Desktop&Mobile':[1,2,3], 'Desktop':[1], 'Mobile':[2,3]}

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth"]

# Указываем список статистик для расчета
statistics = ['reach']

## Создаем словарь с нужными проектами

Для этого подготовим группы и входящие в них проекты в файле Excel. Для примера приведен файл в папке с ноутбуками: __groups.xlsx__

__ВАЖНО!__ Словарь нужно сохранить в папку с ноутбуками
 
__При изменении названия файла/ листа/ колонок, следует внести изменения в код ниже - указать правильное название листа и колонок.__

In [ ]:
sostav = pd.read_excel('groups.xlsx', sheet_name='группы')[['Группа', 'Для словаря']]

pr_groups = {}

for _, t in sostav.iterrows():
    if t['Группа'] not in pr_groups:
        pr_groups[t['Группа']] = f"{t['Для словаря']} OR "
    else:
        pr_groups[t['Группа']] += f"{t['Для словаря']} OR "
for key, value in pr_groups.items():
    pr_groups[key] = value[:-4]
print('Словарь готов')

## Расчёт Total по группе проектов

In [ ]:
%%time

# Посчитаем данные в циклах
tasks = []
print('Отправка задач ')

for utype, num in ut.items():
    for group, filter_obj in pr_groups.items():
        
        # Формируем задание на расчет
        usetype_filter = num
        project_name = f"{group}" 
        mart_filter = f"{filter_obj}"

        # Формируем задание для API Cross Web в формате JSON
        task_json = mtask.build_task('meida', project_name, date_filter, usetype_filter, geo_filter,
                                     demo_filter, mart_filter, slices, statistics)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['usetype'] = f"{utype}"
        tsk['task'] = mtask.send_audience_task(task_json)
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '') 

print('')
        
# Ждем выполнения
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')        
        
# Получаем результат
results = []
for t in tasks:
    tsk = t['task']
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])
    df_result['usetype'] = t['usetype']
    results.append(df_result)
df_result = pd.concat(results)
df_result       

In [ ]:
# Сохраняем в Excel
writer = pd.ExcelWriter(mtask.get_excel_filename('groups-of-projects'))
df_result.to_excel(writer, 'Report', index=False)
writer.save()